## Deploying the knowledge retriever

In [1]:
import tarfile

import sagemaker
import torch
import torch.nn as nn
from sagemaker import get_execution_role
from sagemaker.pytorch import PyTorch as PyTorchEstimator, PyTorchModel

### Set-up

In [6]:
session = sagemaker.Session()
role = get_execution_role()

In [8]:
artifact = 's3://kbchatter/simple/model.tar.gz'

### Deploy the model from S3 (using code in code/inference.py)

In [16]:
pytorch_model = PyTorchModel(model_data=artifact,
                             role=role,
                             framework_version='1.6.0',
                             py_version='py3',
                             entry_point='knowledge-base-chatter/awsdeployment/code/predict.py',
                             source_dir='knowledge-base-chatter/awsdeployment/code',
                             dependencies=[
                                 'knowledge-base-chatter/awsdeployment/code/contexts.json',
                                 'knowledge-base-chatter/models/retrievalmodel.py'  # not in dir for some reason
                             ],
                             )

In [17]:
predictor = pytorch_model.deploy(instance_type='ml.p3.2xlarge',  # 'ml.c4.xlarge',
                                 initial_instance_count=1,
                                 wait=True)

-------------------!

In [18]:
predictor.endpoint  # for lambda

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


'pytorch-inference-2021-07-08-12-26-56-007'

In [19]:
predictor.serializer = sagemaker.serializers.JSONSerializer()
predictor.deserializer = sagemaker.deserializers.JSONDeserializer()

In [23]:
print(predictor.predict({'question': 'How do I delete a channel in Slack?'}))  # need to convert output/input to json for lambda

['how run an executive question and answer ( ama ) session to run an executive ama in slack : create a new channel called # exec - ama. announce it in channels with your largest membership and encourage everyone to join. post a policy document to help guide employees in formatting their questions, and pin it to the channel. set a helpful tone with', 'how do i delete a channel in slack? [SEP] whats send information into a google sheet handy for? : keep track of the information collected by your workflow by automatically sending it to a google sheet. this might come in handy for things like tracking help desk requests, collecting continuous feedback, or managing a nomination process. a few uses for this workflow : sales : gathering ongoing customer feedbackproduct : collecting product feedbackmarketing : tracking incoming requeststeam leaders : archiving employee feedback, recognition, and projectsanyone can trigger this workflow through the shortcuts menu, which then prompts them to fil

## Delete the model and endpoint

In [24]:
pytorch_model.delete_model()

In [25]:
predictor.delete_endpoint()